In [1]:
import spacy
from torchtext import data
import torch
import torch.nn as nn

In [2]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        embedded = self.embedding(x)
        
        output, hidden = self.rnn(embedded)
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [4]:
# load vocabulary
PATH1 = "vocab_naive.pt"
comment = torch.load(PATH1)

PATH2 = "model_naive.pt"
INPUT_DIM = len(comment.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

# create model object
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

# load saved model and evaluate
model.load_state_dict(torch.load(PATH2))
model.eval()

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [5]:
NLP = spacy.load('en')
def predict_sentiment(sentence):
    tokenized = [tok.text for tok in NLP.tokenizer(sentence)] # tokenize input sentence
    indexed = [comment.vocab.stoi[t] for t in tokenized]      # index the tokens
    tensor = torch.LongTensor(indexed)                        # convert tokens to PyTorch tensors
    tensor = tensor.unsqueeze(1)                              # add batch dimension
    prediction = torch.sigmoid(model(tensor))                 # apply on linearity
    return prediction.item()                                  # converts tensor to af float

In [7]:
sent = "I hate you"
predict_sentiment(sent)

0.5476111173629761